# Dotless Arabic Text to Dotted text

## Tokenization in Chars Level

### Load Data

In [ ]:
import pandas as pd
file_path = "C:/Users/lenovo/Desktop/ML-Project/ml_project/src/cleaned_data.csv"
data = pd.read_csv(file_path)
data = data[:100000]
data.head()

In [ ]:
from CharLevelTokenizer import tokenize_char_level
input_sequences, target_sequences, vocab_size, latent_dim, max_length, tokenizer = tokenize_char_level(data)


## BPE Tokenization

### Load Data

In [ ]:
import pandas as pd
file_path = "C:/Users/lenovo/Desktop/ML-Project/ml_project/src/cleaned_data.csv"
data = pd.read_csv(file_path)
data = data[:100000]
data.head()

In [ ]:
from BPETokenizer import BPE_tokenize_data
# Exemple d'utilisation
input_sequences, target_sequences, vocab_size, latent_dim, max_length, tokenizer = BPE_tokenize_data(data)


## Model building 

In [ ]:
from GRU_model import create_seq2seq_model

model = create_seq2seq_model(vocab_size, max_length, latent_dim)
model.summary()


### Training

In [ ]:
history = model.fit([input_sequences, input_sequences], target_sequences,
          batch_size=32,
          epochs=10,
          validation_split=0.2,
            
                   )


In [ ]:
import re

# Fonction pour prédire à partir d'une séquence sans points
def predict_with_lstm(input_text):
    input_seq = tokenizer.texts_to_sequences(['s ' + input_text + ' e'])
    padded_seq = pad_sequences(input_seq, padding='post', maxlen=max_length)
    pred = model.predict([padded_seq, padded_seq])
    pred_text = tokenizer.sequences_to_texts(np.argmax(pred, axis=-1))[0]
    pred_text = pred_text.replace("s", "")
    pred_text = pred_text.replace("e", "")

    return pred_text

# Exemple d'utilisation
input_text = "لدا ڡالدى ٮحارٮ الامارٮعٮه و ٮٮهكم علٮها ڡهو"
predicted_text = predict_with_lstm(input_text)
print(predicted_text)

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()


In [ ]:
# Sauvegarde du modèle complet au format HDF5
model.save('tk_char.keras')

In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model('tk_char.keras')

loaded_model.summary()

In [ ]:
import re

# Fonction pour prédire à partir d'une séquence sans points
def predict_with_lstm(input_text):
    input_seq = tokenizer.texts_to_sequences(['s ' + input_text + ' e'])
    padded_seq = pad_sequences(input_seq, padding='post', maxlen=max_length)
    pred = loaded_model.predict([padded_seq, padded_seq])
    pred_text = tokenizer.sequences_to_texts(np.argmax(pred, axis=-1))[0]
    pred_text = pred_text.replace("s", "")
    pred_text = pred_text.replace("e", "")

    return pred_text

# Exemple d'utilisation
input_text = "ٯڡ"
predicted_text = predict_with_lstm(input_text)
print(predicted_text)